In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 1. 数据读取与预处理（广州数据） ----------------------
# 读取以广州为前缀的最新清洗数据
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_guangzhou_cleaned_20251114_220208")]
if not filtered_files:
    print("❌ No Guangzhou filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded Guangzhou data: {latest_file} (Total: {total} merchants in Guangzhou)")

# 提取有效价格+统计缺失数据（逻辑与北京一致）
def extract_valid_price(price_val):
    if pd.isna(price_val) or str(price_val).strip() in ['人均 -', '', 'None']:
        return None
    match = re.search(r'(\d+)', str(price_val))
    return float(match.group()) if match else None

df['valid_price'] = df['avg_price'].apply(extract_valid_price)
valid_data = df[df['valid_price'].notna()]['valid_price']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据准备 ----------------------
price_bins = [0, 500, 800, 1200, 1500, float('inf')]
price_labels = ['0-500 RMB', '501-800 RMB', '801-1200 RMB', '1201-1500 RMB', '>1500 RMB', 'Missing Data']
price_groups = pd.cut(valid_data, bins=price_bins, labels=price_labels[:-1], right=False)
price_counts = price_groups.value_counts().sort_index()
price_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制广州柱状图（绿色系渐变） ----------------------
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 绿色系配色（从浅到深渐变，最高区间用最深绿，缺失数据用淡绿灰）
colors = [
    '#e8f5e9',  # 最浅绿（0-500 RMB）
    '#c8e6c9',  # 浅绿（501-800 RMB）
    '#a5d6a7',  # 中绿（801-1200 RMB）
    '#66bb6a',  # 深绿（1201-1500 RMB）
    '#2e7d32',  # 最深绿（>1500 RMB，最高区间）
    '#e0e0d1'   # 淡绿灰（缺失数据，带绿色调的浅灰）
]

fig, ax = plt.subplots(figsize=(12, 7))
bars = ax.bar(
    price_counts.index, 
    price_counts.values, 
    color=colors,  # 应用绿色系渐变
    width=0.65,
    edgecolor='white', 
    linewidth=1.5, 
    alpha=0.9  # 轻微透明，避免颜色过饱和
)

# 数据标签：显示数量+百分比（与北京样式一致）
for bar in bars:
    height = bar.get_height()
    percentage = (height / total) * 100
    ax.text(
        bar.get_x() + bar.get_width()/2, 
        height + 0.5,
        f"{int(height)}\n({percentage:.1f}%)",
        ha='center', 
        va='bottom', 
        fontsize=9, 
        fontweight='bold', 
        color='#2e3b2f'  # 深绿灰文字，与绿色系对比清晰
    )

# 标题：明确标注Guangzhou，字体风格与北京一致
ax.set_title(
    'Per Capita Price Distribution of Pet Funeral Merchants in Guangzhou',
    fontsize=16, 
    fontweight='bold', 
    color='#2e7d32'  # 标题色与最深绿呼应，强化广州识别度
)
ax.set_xlabel('Price Range', fontsize=12, fontweight='medium')
ax.set_ylabel('Number of Merchants', fontsize=12, fontweight='medium')

# 美化细节（与北京图表统一风格）
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=10, rotation=30)  # x轴标签轻微旋转，避免重叠
ax.tick_params(axis='y', labelsize=10)
ax.set_ylim(0, max(price_counts.values) * 1.2)  # 预留顶部空间显示标签

# ---------------------- 4. 保存图片 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'guangzhou_price_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Guangzhou chart saved to: {save_path}")

✅ Loaded Guangzhou data: pet_funeral_guangzhou_cleaned_20251114_220208.xlsx (Total: 62 merchants in Guangzhou)
📁 Guangzhou chart saved to: /Users/syx/Documents/python脚本/guangzhou_price_distribution_20251114_231839.png


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 配色方案（广州专属绿色系） ----------------------
colors = [
    '#e8f5e9',  # 最浅绿（<3.5）
    '#c8e6c9',  # 浅绿（3.5-4.0）
    '#a5d6a7',  # 中绿（4.0-4.5）
    '#2e7d32',  # 最深绿（4.5-5.0）
    '#e0e0d1'   # 淡绿灰（缺失数据）
]

# ---------------------- 1. 数据读取与预处理 ----------------------
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_guangzhou_cleaned_20251114_220208")]
if not filtered_files:
    print("❌ No Guangzhou filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded Guangzhou data: {latest_file} (Total: {total} merchants in Guangzhou)")

def extract_valid_rating(star_val):
    if pd.isna(star_val) or str(star_val).strip() in ['0.0', '0.0分', '无评分', '', 'None']:
        return None
    match = re.search(r'(\d+\.?\d*)', str(star_val))
    return float(match.group()) if match and float(match.group()) > 0 else None

df['valid_rating'] = df['star'].apply(extract_valid_rating)
valid_data = df[df['valid_rating'].notna()]['valid_rating']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据分组 ----------------------
rating_bins = [0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ['<3.5(Poor)', '3.5-4.0(Fair)', '4.0-4.5(Good)', '4.5-5.0 (Excellent)', 'Missing Data']
rating_groups = pd.cut(valid_data, bins=rating_bins, labels=rating_labels[:-1], right=False)
rating_counts = rating_groups.value_counts().sort_index()
rating_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制饼图（优化图例布局） ----------------------
plt.style.use('seaborn-v0_8-white')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']

fig, ax = plt.subplots(figsize=(9, 9))
wedgeprops = {'edgecolor': 'white', 'linewidth': 2}

wedges, texts, autotexts = ax.pie(
    rating_counts.values,
    labels=rating_labels,
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    pctdistance=0.82,
    labeldistance=1.1,
    wedgeprops=wedgeprops,
    textprops={'fontsize': 11}
)

# 美化标签
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
for text in texts:
    text.set_fontweight('bold')
    text.set_color('#2e3b2f')

# 标题
ax.set_title(
    'Distribution of Pet Funeral Merchant Ratings in Guangzhou',
    fontsize=16, 
    fontweight='bold', 
    color='#2e7d32'
)

# 【核心优化】图例分2列显示，底部居中布局，避免重叠
legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]
ax.legend(
    legend_labels, 
    loc='center',  # 定位到中心
    bbox_to_anchor=(0.5, -0.1),  # 调整到底部居中
    ncol=2,  # 分2列显示
    fontsize=10,
    frameon=True,
    edgecolor='#c8e6c9'
)

# ---------------------- 4. 保存图片 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'guangzhou_rating_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Guangzhou rating chart saved to: {save_path}")

✅ Loaded Guangzhou data: pet_funeral_guangzhou_cleaned_20251114_220208.xlsx (Total: 62 merchants in Guangzhou)


/var/folders/9q/ywtdr2kx78d4gg56pbn1dk_w0000gn/T/ipykernel_46765/327485810.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]


📁 Guangzhou rating chart saved to: /Users/syx/Documents/python脚本/guangzhou_rating_distribution_20251114_232206.png


In [13]:
import pandas as pd
import os
from pyecharts import options as opts
from pyecharts.charts import Map
from datetime import datetime

# ---------------------- 配置路径 ----------------------
INPUT_CLEANED_DATA = "/Users/syx/Documents/python脚本/guangzhou_second_cleaned_20251114.xlsx"  # 广州清洗文件路径
OUTPUT_HEATMAP = f"广州宠物店铺热力图_{datetime.now().strftime('%Y%m%d')}.html"


# ---------------------- 广州所有行政区列表 ----------------------
all_guangzhou_districts = [
    "越秀区", "海珠区", "荔湾区", "天河区", "白云区", 
    "黄埔区", "南沙区", "番禺区", "花都区", "增城区", "从化区"
]


# ---------------------- 生成热力图（标记点颜色与绿色系渐变统一） ----------------------
if os.path.exists(INPUT_CLEANED_DATA):
    df = pd.read_excel(INPUT_CLEANED_DATA)
    if "district_cn" in df.columns:
        district_counts = df["district_cn"].value_counts().to_dict()
        for dist in all_guangzhou_districts:
            if dist not in district_counts:
                district_counts[dist] = 0
        heatmap_data = [(k, v) for k, v in district_counts.items()]

        # 绿色渐变配色（与标记点、区域颜色完全统一）
        green_gradient = [
            "#f1f8e9", "#e8f5e9", "#dcedc1", "#c5e1a5",
            "#aed581", "#9ccc65", "#8bc34a", "#689f38"
        ]

        c = (
            Map(
                init_opts=opts.InitOpts(
                    width="1200px",
                    height="700px",
                    page_title="Guangzhou Pet Funeral Heatmap"
                )
            )
            .add(
                series_name="Number of Merchants",
                data_pair=heatmap_data,
                maptype="广州",
                label_opts=opts.LabelOpts(is_show=True, font_size=10, color="#333"),
                # 关键：标记点颜色与热力图渐变统一
                itemstyle_opts=opts.ItemStyleOpts(
                    color=green_gradient,  # 标记点颜色继承渐变列表
                    border_color="#fff",
                    border_width=1
                )
            )
            .set_global_opts(
                title_opts=opts.TitleOpts(
                    title="Distribution of Pet Funeral Stores in Guangzhou",
                    pos_left="center",
                    pos_top="15px",
                    title_textstyle_opts=opts.TextStyleOpts(
                        font_size=18, color="#689f38", font_weight="bold"
                    )
                ),
                legend_opts=opts.LegendOpts(
                    pos_bottom="10px",
                    pos_left="center",
                    textstyle_opts=opts.TextStyleOpts(color="#8bc34a")
                ),
                visualmap_opts=opts.VisualMapOpts(
                    min_=0,
                    max_=max([v for _, v in heatmap_data]) if heatmap_data else 1,
                    range_color=green_gradient,
                    pos_right="20px",
                    pos_top="center",
                    orient="vertical",
                    is_calculable=True,
                    textstyle_opts=opts.TextStyleOpts(color="#8bc34a")
                ),
                toolbox_opts=opts.ToolboxOpts(
                    is_show=True,
                    feature={
                        "zoom": {"show": True, "title": "放大"},
                        "restore": {"show": True, "title": "重置"}
                    },
                    pos_right="30px",
                    pos_top="50px",
                    item_size=15
                )
            )
            .set_series_opts(
                tooltip_opts=opts.TooltipOpts(
                    formatter=lambda params: f"""
                    <div style="text-align:center; min-width:100px;">
                        <div style="font-weight:bold; color:#689f38;">{params.name}</div>
                        <div style="color:#8bc34a;">{
                            "暂无店铺" if params.value == 0 
                            else f"Number of Merchants：{params.value}"
                        }</div>
                    </div>
                    """,
                    background_color="rgba(241, 248, 233, 0.9)",
                    border_color="#aed581",
                    textstyle_opts=opts.TextStyleOpts(color="#689f38")
                )
            )
        )

        c.render(OUTPUT_HEATMAP)
        print(f"✅ 广州热力图已保存：{OUTPUT_HEATMAP}")
    else:
        print("❌ 数据中无 'district_cn' 列，请检查清洗步骤")
else:
    print(f"❌ 未找到文件：{INPUT_CLEANED_DATA}")

✅ 广州热力图已保存：广州宠物店铺热力图_20251114.html
